## Pytorch gradient test
Check how gradient calculation is working in mid-layers.
Below stream is,

$$x \xrightarrow{func(x)} z \xrightarrow{func2(z)} y$$

I want to know $$\frac{\partial y}{\partial z}$$, and $$\frac{\partial y}{\partial x}$$.

First, import torch and define input 1.

In [8]:
import torch

x = torch.ones(1, requires_grad = True)

or do like,

In [9]:
x = torch.ones(1)
x.requires_grad_(True)

tensor([1.], requires_grad=True)

Next, define `grad()`.
`tensor.register_hook(grad)` give the tensor's gradient to grad() as param.

In [10]:
# saved gradients. stacked from back.
grads = []

# save gradients.
def grad(grad):
    grads.append(grad)

Next, define $$func(x)$$ and $$func2(z)$$.  
And regist hook.  

$$func(x) = 2x$$  

$$func2(z) = z^2 + 2z + 2$$  

In [11]:
def func(x):
    z = 2 * x
    z.register_hook(grad)
    return z

def func2(z):
    y = (z ** 2) + (2 * z) + 2
    y.register_hook(grad)
    return y

x.register_hook(grad)

Next, feed x to $$func(x)$$ and $$func2(z)$$.

In [12]:
z = func(x)
y = func2(z)

Next, calculate gradient!
After Tensor.backward(), gradients of x ,z will be saved at `grads`.

In [13]:
y.backward()

See grads.

In [14]:
print(grads)

[tensor([1.]), tensor([6.]), tensor([12.])]


Because of backward(), gradients are saved from backside.
$$grad[0] = \frac{\partial y}{\partial y}$$

$$grad[1] = \frac{\partial y}{\partial z}$$

$$grad[2] = \frac{\partial y}{\partial x}$$

First, $$\frac{\partial y}{\partial y} = 1$$,

Next, $$\frac{\partial y}{\partial z} = 2z+2$$.
Because of $$func(x)$$, input of $$func2(z)$$ is 2 ($$2 \times 1$$),
So, $$\frac{\partial y}{\partial z} = 6$$

Finally, $$\frac{\partial y}{\partial x} = \frac{\partial y}{\partial z}\frac{\partial z}{\partial x}$$,
$$\frac{\partial y}{\partial z} = 2z+2$$ and $$\frac{\partial z}{\partial x} =2$$
So, $$\frac{\partial y}{\partial x} = 4z+4$$,
Because of $$func(x)$$, $$z = 2x = 2$$,
So,  $$\frac{\partial y}{\partial x} = 4 \times 2 + 4 = 12$$

### Result
So if we want to know gradient of mid layers output $$n$$ and value $$m$$, $$\frac{\partial m}{\partial n}$$,
Use `m.backward()` and `n.register_hook(grad)`